# Imports

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

import requests
import json

import altair as alt

In [2]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [3]:
PROJECT_DIR = '../'
DATA_DIR = PROJECT_DIR + 'data/'
REPORTS_DIR = PROJECT_DIR + 'reports/'

# Data exploration

### A) Create the map

In [4]:
json_file = DATA_DIR + 'departements.geojson'
df_map = gpd.read_file(json_file)
df_map.columns = ['department_id', 'department_name', 'geometry']
df_map

,department_id,department_name,geometry
0,02,Aisne,"POLYGON ((3.17270 50.01200, 3.18220 50.01234, ..."
1,10,Aube,"POLYGON ((3.41479 48.39027, 3.42208 48.41334, ..."
2,14,Calvados,"POLYGON ((-1.11962 49.35557, -1.11503 49.36240..."
3,15,Cantal,"POLYGON ((2.50841 45.47850, 2.52444 45.48070, ..."
4,28,Eure-et-Loir,"POLYGON ((0.81482 48.67016, 0.82767 48.68072, ..."
...,...,...,...
91,41,Loir-et-Cher,"POLYGON ((0.84122 48.10306, 0.87588 48.10943, ..."
92,53,Mayenne,"POLYGON ((-0.86036 48.50146, -0.84611 48.49828..."
93,54,Meurthe-et-Moselle,"MULTIPOLYGON (((5.45556 49.48093, 5.44953 49.4..."
94,79,Deux-Sèvres,"POLYGON ((-0.89196 46.97582, -0.87973 46.97580..."


### B) Load baby names information

In [5]:
data_file = DATA_DIR + 'dpt2020.csv'
df_w_sex = pd.read_csv(data_file, delimiter=';')
df_w_sex.columns = ['sex', 'name', 'year', 'department_id', 'count']
df_w_sex  

,sex,name,year,department_id,count
0,1,_PRENOMS_RARES,1900,02,7
1,1,_PRENOMS_RARES,1900,04,9
2,1,_PRENOMS_RARES,1900,05,8
3,1,_PRENOMS_RARES,1900,06,23
4,1,_PRENOMS_RARES,1900,07,9
...,...,...,...,...,...
3727548,2,ZYA,2018,59,3
3727549,2,ZYA,XXXX,XX,264
3727550,2,ZYNA,2013,93,3
3727551,2,ZYNA,XXXX,XX,59


In [6]:
df_w_sex['name'] = df_w_sex['name'].replace('_PRENOMS_RARES', np.nan)
df_w_sex['year'] = df_w_sex['year'].replace('XXXX', np.nan)
df_w_sex = df_w_sex.dropna()
df_w_sex

,sex,name,year,department_id,count
10885,1,AADIL,1983,84,3
10886,1,AADIL,1992,92,3
10888,1,AAHIL,2016,95,3
10892,1,AARON,1962,75,3
10893,1,AARON,1976,75,3
...,...,...,...,...,...
3727545,2,ZYA,2013,44,4
3727546,2,ZYA,2013,59,3
3727547,2,ZYA,2017,974,3
3727548,2,ZYA,2018,59,3


In [7]:
df_wo_sex = df_w_sex.groupby(by=['name', 'year', 'department_id']).sum().reset_index()
df_wo_sex

,name,year,department_id,sex,count
0,AADIL,1983,84,1,3
1,AADIL,1992,92,1,3
2,AAHIL,2016,95,1,3
3,AALIYA,2017,75,2,3
4,AALIYAH,2001,92,2,4
...,...,...,...,...,...
3637581,ÖMER,2020,45,1,3
3637582,ÖMER,2020,60,1,4
3637583,ÖMER,2020,67,1,3
3637584,ÖMER,2020,68,1,3


In [8]:
display(df_w_sex['name'].value_counts())
print()
display(df_wo_sex['name'].value_counts())

CAMILLE        13820
MARIE          13300
PIERRE         11388
PAUL           10711
JEAN           10694
               ...  
FELINE             1
ROBESPIERRE        1
MOMINA             1
LEONCINE           1
ZYNA               1
Name: name, Length: 15270, dtype: int64

MARIE     11353
PIERRE    11278
PAUL      10706
JEAN      10639
LOUIS     10119
          ...  
AÏRON         1
AÏTOR         1
MATTEI        1
BADICE        1
BARTHA        1
Name: name, Length: 15270, dtype: int64

### C) Create the final dataframe

# Data visualization

### A) Visualization 1

##### Top names of all time

In [9]:
df_w_sex_france = df_w_sex.groupby(by=['name', 'sex']).sum().reset_index()
df_w_sex_france['sex']= df_w_sex_france['sex'].map({1:'man', 2:'woman'})
df_w_sex_france

C:\Users\louis\AppData\Local\Temp\ipykernel_16224\756159227.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_w_sex_france = df_w_sex.groupby(by=['name', 'sex']).sum().reset_index()


,name,sex,count
0,AADIL,man,6
1,AAHIL,man,3
2,AALIYA,woman,3
3,AALIYAH,woman,1794
4,AALYA,woman,6
...,...,...,...
16133,ÉZÉCHIEL,man,45
16134,ÉZÉKIEL,man,9
16135,ÎMRAN,man,3
16136,ÏSSA,man,3


In [10]:
# Select top_n names
top_n_names = 50

# Retrieve the top 50 names
top_names = df_w_sex_france.groupby(by=['name']).sum().reset_index('name').sort_values('count', ascending=False)['name'].values[:top_n_names]
print(top_names)

# Filter dataframe
df_w_sex_france_filtered = df_w_sex_france[df_w_sex_france['name'].isin(top_names)]
df_w_sex_france_filtered

['MARIE' 'JEAN' 'PIERRE' 'MICHEL' 'ANDRÉ' 'JEANNE' 'PHILIPPE' 'LOUIS'
 'RENÉ' 'ALAIN' 'JACQUES' 'BERNARD' 'MARCEL' 'CLAUDE' 'DANIEL' 'ROGER'
 'PAUL' 'ROBERT' 'DOMINIQUE' 'GEORGES' 'HENRI' 'CHRISTIAN' 'NICOLAS'
 'FRANÇOISE' 'MONIQUE' 'FRANÇOIS' 'PATRICK' 'CATHERINE' 'NATHALIE'
 'GÉRARD' 'ISABELLE' 'JOSEPH' 'CHRISTOPHE' 'JACQUELINE' 'ANNE' 'SYLVIE'
 'JULIEN' 'MAURICE' 'LAURENT' 'FRÉDÉRIC' 'MARTINE' 'ERIC' 'DAVID'
 'STÉPHANE' 'PASCAL' 'MADELEINE' 'SÉBASTIEN' 'ALEXANDRE' 'NICOLE'
 'THIERRY']


C:\Users\louis\AppData\Local\Temp\ipykernel_16224\1296162584.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  top_names = df_w_sex_france.groupby(by=['name']).sum().reset_index('name').sort_values('count', ascending=False)['name'].values[:top_n_names]


,name,sex,count
446,ALAIN,man,504103
447,ALAIN,woman,3
537,ALEXANDRE,man,299066
538,ALEXANDRE,woman,24
970,ANDRÉ,man,709568
...,...,...,...
14481,SYLVIE,woman,361407
14508,SÉBASTIEN,man,301409
14509,SÉBASTIEN,woman,38
14824,THIERRY,man,287597


In [11]:
# Create the brush filter
# brush = alt.selection_interval(encodings=['x'])
default_name = 'JEAN'
# brush = alt.selection_single(name='name', fields=['name'], bind='name', value={'name': default_name})
# brush = alt.selection_single(name='name', fields=['name'])
# brush_year = alt.binding_select(options=top_names, name='Name:')
# brush = alt.selection_point(name='name_selector', fields=['name'], value=default_name, bind=brush_year)
brush = alt.selection_point(name='name_selector', fields=['name'], value=default_name)

# Create the global rank and the sex rank
s_rank_global = df_w_sex_france_filtered.groupby(by=['name']).sum()['count'].rank(ascending=False, method='min').astype(int)
df_w_sex_france_filtered['rank_global'] = df_w_sex_france_filtered['name'].apply(lambda x: s_rank_global.loc[x])
df_w_sex_france_filtered['rank_sex'] = df_w_sex_france_filtered.groupby('sex')['count'].rank(ascending=False, method='min').astype(int)

# Top best names count
top_names_bar_chart = alt.Chart(df_w_sex_france_filtered).mark_bar().encode(
    alt.X('name:N').sort('-y'),
    alt.Y('count:Q'),
    alt.Color('sex:N', scale=alt.Scale(domain=['man', 'woman'], range=['blue', 'red'])),
    tooltip = [alt.Tooltip('rank_global:Q', title='Rank (global)'),
               alt.Tooltip('rank_sex:N', title='Rank (sex)'),
               alt.Tooltip('name:N', title='Name'),
               alt.Tooltip('sex:N', title='Sex'),
               alt.Tooltip('count:Q', title='Count')
    ],
    opacity=alt.condition(brush, alt.value(1), alt.value(0.2)),
).properties(
    title=alt.TitleParams(text=f"Most common {top_n_names} baby names of all time in France", fontSize=20),
    width=800,
    height=200
).add_params(
    brush
)

# Display the bar chart
top_names_bar_chart

C:\Users\louis\AppData\Local\Temp\ipykernel_16224\293578643.py:11: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  s_rank_global = df_w_sex_france_filtered.groupby(by=['name']).sum()['count'].rank(ascending=False, method='min').astype(int)
C:\Users\louis\AppData\Local\Temp\ipykernel_16224\293578643.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_w_sex_france_filtered['rank_global'] = df_w_sex_france_filtered['name'].apply(lambda x: s_rank_global.loc[x])
C:\Users\louis\AppData\Local\Temp\ipykernel_16224\293578643.py:13: SettingWithCopyWarning: 
A value 

alt.Chart(...)

##### Top names over time

In [12]:
df_wo_sex_france = df_wo_sex.groupby(by=['name', 'sex', 'year']).sum().reset_index()
df_wo_sex_france['sex'] = df_wo_sex_france['sex'].map({1:'man', 2:'woman', 3:'both'})
df_wo_sex_france

C:\Users\louis\AppData\Local\Temp\ipykernel_16224\3976557541.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_wo_sex_france = df_wo_sex.groupby(by=['name', 'sex', 'year']).sum().reset_index()


,name,sex,year,count
0,AADIL,man,1983,3
1,AADIL,man,1992,3
2,AAHIL,man,2016,3
3,AALIYA,woman,2017,3
4,AALIYAH,woman,2001,9
...,...,...,...,...
257843,ÖMER,man,2016,18
257844,ÖMER,man,2017,30
257845,ÖMER,man,2018,31
257846,ÖMER,man,2019,37


In [13]:
# Filter dataframe
df_w_sex_filtered = df_w_sex[df_w_sex['name'].isin(top_names[:10])]
df_w_sex_filtered = df_w_sex_filtered.groupby(by=['name', 'sex', 'year']).sum().reset_index(['name', 'sex', 'year'])
df_w_sex_filtered['sex'] = df_w_sex_filtered['sex'].map({1:'man', 2:'woman'})
df_w_sex_filtered

C:\Users\louis\AppData\Local\Temp\ipykernel_16224\726164686.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_w_sex_filtered = df_w_sex_filtered.groupby(by=['name', 'sex', 'year']).sum().reset_index(['name', 'sex', 'year'])


,name,sex,year,count
0,ALAIN,man,1900,83
1,ALAIN,man,1901,99
2,ALAIN,man,1902,106
3,ALAIN,man,1903,120
4,ALAIN,man,1904,136
...,...,...,...,...
1437,RENÉ,man,2002,3
1438,RENÉ,man,2006,3
1439,RENÉ,man,2015,3
1440,RENÉ,woman,1904,3


In [14]:
# Line chart for men
line_men = alt.Chart(df_w_sex_filtered).mark_line().encode(
    alt.X('year:N', scale=alt.Scale(zero=False), axis=alt.Axis(tickCount=10, labelExpr='parseInt(datum.value) % 5 === 0 ? datum.label : ""')),
    alt.Y('count:Q', scale=alt.Scale(zero=False)),
    alt.Color('name:N', legend=alt.Legend(title='Name')),
    alt.Shape('sex:N', scale=alt.Scale(domain=['woman', 'man'], range=['triangle', 'square']), legend=alt.Legend(title='Sex')),
    tooltip=[alt.Tooltip('year:N', title='Year'),
             alt.Tooltip('name:N', title='Name'),
             alt.Tooltip('sex:N', title='Sex'),
             alt.Tooltip('count:Q', title='Count')
    ],
).transform_filter(
    alt.datum.sex == 'man'
).properties(
    title=alt.TitleParams(text=f"Evolution of the baby names in France over time", fontSize=20),
    width=800,
    height=400
)

# Line chart for women
line_women = alt.Chart(df_w_sex_filtered).mark_line().encode(
    alt.X('year:N', scale=alt.Scale(zero=False), axis=alt.Axis(tickCount=10, labelExpr='parseInt(datum.value) % 5 === 0 ? datum.label : ""')),
    alt.Y('count:Q', scale=alt.Scale(zero=False)),
    alt.Color('name:N', legend=alt.Legend(title='Name')),
    alt.Shape('sex:N', scale=alt.Scale(domain=['woman', 'man'], range=['triangle', 'square']), legend=alt.Legend(title='Sex')),
    tooltip=[alt.Tooltip('year:N', title='Year'),
             alt.Tooltip('name:N', title='Name'),
             alt.Tooltip('sex:N', title='Sex'),
             alt.Tooltip('count:Q', title='Count')
    ],
).transform_filter(
    alt.datum.sex == 'woman'
).properties(
    title=alt.TitleParams(text=f"Evolution of the baby names in France over time", fontSize=20),
    width=800,
    height=400
)

# Combine the line charts
line = alt.layer(line_men, line_women).resolve_scale(shape='independent').properties(
    title=alt.TitleParams(text=f"Evolution of the baby names in France over time", fontSize=20),
    width=1000,
    height=400
)

## Create a selection that chooses the nearest point & selects based on x-value
# nearest = alt.selection_point(nearest=True, on='mouseover', fields=['year'], empty=False)

# # Transparent selectors across the chart. This is what tells us the x-value of the cursor
# selectors = alt.Chart(df_w_sex_filtered).mark_point().encode(
#     x='year:N',
#     opacity=alt.value(0),
# ).add_params(
#     nearest
# )

# # Draw points on the line, and highlight based on selection
# points_men = line_men.mark_point().encode(opacity=alt.condition(nearest, alt.value(1), alt.value(0)))
# points_women = line_women.mark_point().encode(opacity=alt.condition(nearest, alt.value(1), alt.value(0)))

# # Draw text labels near the points, and highlight based on selection
# text_men = line_men.mark_text(align='left', dx=5, dy=-5).encode(text=alt.condition(nearest, 'count:Q', alt.value(' ')))
# text_women = line_women.mark_text(align='left', dx=5, dy=-5).encode(text=alt.condition(nearest, 'count:Q', alt.value(' ')))

# # Draw a rule at the location of the selection
# rules = alt.Chart(df_w_sex_filtered).mark_rule(color='gray').encode(
#     x='year:N',
# ).transform_filter(
#     nearest
# )

# # Display the line chart
# line_chart = alt.layer(
#     line, selectors, points_men, points_women, rules, text_men, text_women
# )
# line_chart

line_men_women = line_men + line_women
line_men_women = line_men_women#.add_params(brush).transform_filter(brush)
line_men_women

alt.LayerChart(...)

In [15]:
# # line_chart = line_men.resolve_scale(y='independent')
# # line_chart = line_men.properties(height=200)

# # Combine the charts using layer()
# # combined_chart = alt.layer(top_names_bar_chart, line_chart)
# # combined_chart = alt.vconcat(alt.layer(top_names_bar_chart), line_chart)

# combined_chart = (top_names_bar_chart & line_men_women)
# combined_chart

### B) Visualization 2

##### Evolution of baby names over time (map)

In [16]:
# target_name = 'JEAN'

# df_target_name = df_wo_sex[df_wo_sex['name'] == target_name].reset_index(drop=True)
# df_target_name['year'] = df_target_name['year'].astype(str)
# df_target_name

# df_target_name_per_year = df_target_name.pivot(index=['department_id', 'name'], columns='year', values='count').reset_index()
# years_to_plot = list(df_target_name_per_year.columns[2:])
# df_target_name_per_year

In [17]:
# df_target_name = df_wo_sex[df_wo_sex['name'] == target_name].reset_index(drop=True)
df_target_name = df_wo_sex.copy()
df_target_name['year'] = df_target_name['year'].astype(str)
df_target_name

df_target_name_per_year = df_target_name.pivot(index=['department_id', 'name'], columns='year', values='count').reset_index()
years_to_plot = list(df_target_name_per_year.columns[2:])
df_target_name_per_year = df_target_name_per_year[df_target_name_per_year['name'].isin(top_names)]
df_target_name_per_year

year,department_id,name,1900,1901,1902,1903,1904,1905,1906,1907,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
27,01,ALAIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,01,ALEXANDRE,10.0,10.0,15.0,13.0,10.0,12.0,10.0,7.0,...,9.0,6.0,10.0,5.0,8.0,NaN,7.0,NaN,NaN,3.0
101,01,ANDRÉ,18.0,32.0,24.0,44.0,43.0,33.0,39.0,60.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121,01,ANNE,21.0,10.0,15.0,10.0,14.0,12.0,6.0,17.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
220,01,BERNARD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234759,974,ROGER,NaN,7.0,4.0,8.0,8.0,13.0,8.0,19.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235050,974,STÉPHANE,4.0,6.0,NaN,5.0,6.0,5.0,10.0,6.0,...,3.0,4.0,3.0,4.0,3.0,NaN,NaN,NaN,NaN,NaN
235082,974,SYLVIE,NaN,NaN,NaN,4.0,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235088,974,SÉBASTIEN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.0,8.0,10.0,5.0,8.0,4.0,NaN,4.0,NaN,NaN


In [ ]:
# Create slider
min_year = int(min(years_to_plot))
max_year = int(max(years_to_plot))
slider_year = alt.binding_range(min=min_year, max=max_year, step=1, name='Year:')
selector_year = alt.selection_point(name='year_selector', fields=['year'], bind=slider_year, value=1900)

# Define color scale
color_scale = alt.Scale(scheme='turbo')

# Create map chart
map_chart = alt.Chart(df_map).mark_geoshape(
    stroke='white',
    strokeWidth=0.05
).transform_lookup(
    lookup='department_id',
    from_=alt.LookupData(df_target_name_per_year, key='department_id', fields=['name']+years_to_plot)
).transform_fold(
    years_to_plot, as_=['year', 'count']
).transform_calculate(
    year='parseInt(datum.year)',
    count='isValid(datum.count) ? datum.count : -0',
    # name=alt.condition(alt.datum.name == default_name, alt.datum.name, alt.value(None))
    name='datum.name'

).encode(
    color=alt.condition(
        predicate='datum.count > 0',
        if_true=alt.Color('count:Q', scale=alt.Scale(scheme='viridis')),
        if_false=alt.value('#dfdbf6')
    ),
    tooltip=[
        alt.Tooltip('department_id:N', title='Department code'),
        alt.Tooltip('department_name:N', title='Department name'),
        alt.Tooltip('name:N', title='Name'),
        alt.Tooltip('count:Q', title='Count'),
        alt.Tooltip('year:Q', title='Year')
    ],
    opacity=alt.condition(brush, alt.value(1), alt.value(0.2)),
).add_params(
    selector_year,
    brush
).transform_filter(
    selector_year
).transform_filter(
    # brush
    alt.datum.name == brush['name'][0]
).project(
    type='conicConformal',
    center=[2.454071, 46.279229],
    scale=4000,
    translate=[500, 400]
).properties(
    title=alt.TitleParams(text=f"Evolution of baby names in France over time", fontSize=20),
    width=1000,
    height=800
)

# Display the map
# line_men & map_chart
top_names_bar_chart & map_chart
# top_names_bar_chart

##### Evolution of baby names over time (bar chart)

In [ ]:
df_w_sex_target_name = df_w_sex[df_w_sex['name']==default_name]
df_w_sex_target_name = df_w_sex_target_name.groupby(by=['sex', 'name', 'year']).sum().reset_index()
df_w_sex_target_name['sex'] = df_w_sex_target_name['sex'].map({1:'man', 2:'woman'})
df_w_sex_target_name

In [ ]:
df_wo_sex_target_name = df_wo_sex[df_wo_sex['name']==default_name]
df_wo_sex_target_name = df_wo_sex_target_name.groupby(by=['name', 'year']).sum().reset_index()
df_wo_sex_target_name['sex'] = 'man + woman'
df_wo_sex_target_name

In [ ]:
### Code adapated from https://altair-viz.github.io/gallery/multiline_tooltip.html


# Create a selection that chooses the nearest point & selects based on x-value
nearest = alt.selection_point(nearest=True, on='mouseover', fields=['year'], empty=False)

# Create line chart (men, women)
line_men_women = alt.Chart(df_w_sex_target_name).mark_line(interpolate='basis').encode(
    alt.X('year:N', scale=alt.Scale(zero=False), axis=alt.Axis(tickCount=10, labelExpr='parseInt(datum.value) % 5 === 0 ? datum.label : ""')),
    alt.Y('count:Q', scale=alt.Scale(zero=False)),
    alt.Color('sex:N', scale=alt.Scale(domain=['woman', 'man'], range=['red', 'blue'])),
    tooltip = [alt.Tooltip('year:N', title='Year'),
               alt.Tooltip('name:N', title='Name'),
               alt.Tooltip('sex:N', title='Sex'),
               alt.Tooltip('count:Q', title='Count')
    ],
)

# Create line chart (both men and women)
line_both = alt.Chart(df_wo_sex_target_name).mark_line(interpolate='basis').encode(
    alt.X('year:N', scale=alt.Scale(zero=False), axis=alt.Axis(tickCount=10, labelExpr='parseInt(datum.value) % 5 === 0 ? datum.label : ""')),
    alt.Y('count:Q', scale=alt.Scale(zero=False)),
    alt.Color('sex:N', scale=alt.Scale(domain=['man + woman'], range=['green'])),
    tooltip = [alt.Tooltip('year:N', title='Year'),
               alt.Tooltip('name:N', title='Name'),
               alt.Tooltip('sex:N', title='Sex'),
               alt.Tooltip('count:Q', title='Count')
    ],
)

# Combine the charts using layer
# line = alt.layer(
#     line_men_women, line_both
# ).properties(
#     title=alt.TitleParams(text=f"Evolution of the baby name '{target_name}' in France over time", fontSize=20),
#     width=1000,
#     height=200
# )

line = line_men_women.properties(
    title=alt.TitleParams(text=f"Evolution of the baby name '{target_name}' in France over time", fontSize=20),
    width=1000,
    height=200
)

# Transparent selectors across the chart. This is what tells us the x-value of the cursor
selectors = alt.Chart(df_w_sex_target_name).mark_point().encode(
    x='year:N',
    opacity=alt.value(0),
).add_params(
    nearest
)

# Draw points on the line, and highlight based on selection
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = line.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'count:Q', alt.value(' '))
)

# Draw a rule at the location of the selection
rules = alt.Chart(df_w_sex_target_name).mark_rule(color='gray').encode(
    x='year:N',
).transform_filter(
    nearest
)

# Display the line chart
line_chart = alt.layer(
    line, selectors, points, rules, text
)
line_chart

# Final dashboard (stacked visualizations)

In [ ]:
top_names_bar_chart & 